In [ ]:
import logging
import datetime
import holidays
from copy import deepcopy
import re

from IPython.core.display import display, HTML

import os
os.chdir('H:/Mes Documents/ServiceCivil2019/schedvisu')
import sys
sys.path.append('scripts')

from scripts.main import load_config, get_day_range
from retrieve_data import retrieve_and_save_data_from_PACS
from extract_data import load_transform_and_save_data_from_files
from create_report import create_report, get_report_type

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

### Load the config and get the series using the API

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-07'
config['main']['end_date'] = '2019-12-06'
df_studies = load_transform_and_save_data_from_files(config)
df_studies

In [ ]:
pd.DataFrame(df_studies.groupby(['Machine', 'Description'])['Patient ID'].count()).sort_values(['Machine', 'Patient ID'], ascending=False)

In [ ]:
df_studies[df_studies['Description'] == 'OTHER'].groupby(['Machine', 'Study Description'])['Patient ID'].count().sort_values(ascending=False)

## Do the loading from a main studies.pkl file and expanding it, instead of recreating the file every time

### 1) get the already processed studies if any

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-01'
config['main']['end_date'] = '2019-01-15'

start_date, end_date, days_range = main.get_day_range(config)
studies_save_path = 'data/studies.pkl'
df_studies = None
already_processed_days = []
holiday_days = holidays.Switzerland(prov='VD')

# check if the data has already been extracted, transformed and saved
if os.path.isfile(studies_save_path):
    logging.info('Reading studies database: save file found at "{}", loading data'.format(studies_save_path))
    df_studies = pd.read_pickle(studies_save_path)
    already_processed_days = set(df_studies['Date'].tolist())

### 2) check whether all required days have already been processed

In [ ]:
days_to_process = [day for day in days_range if day.strftime('%Y%m%d') not in already_processed_days and day.weekday() not in [5, 6] and day not in holiday_days]
days_to_process

### 3) process the days that are not present in the DataFrame

In [ ]:
for day in days_to_process:
    logging.info('Processing {}: day is required but not present in the main studies DataFrame'.format(day.strftime('%Y%m%d'))

    # create a local config object just to process the specified days
    local_config = deepcopy(config)
    local_config['main']['start_date'] = day.strftime('%Y-%m-%d')
    local_config['main']['end_date'] = day.strftime('%Y-%m-%d')
                 
    # load in the data
    df_series = load_data_from_files(local_config)
    # mark the rektakes and the machine group for each series
    df_series = mark_retakes(local_config, df_series)
    df_series = mark_machine_group(local_config, df_series)
    # show some info about the series and studies
    #show_series_groupby(config, df_series)

    # group the series together into a DataFrame of studies
    df_studies_for_day = df_series.replace(np.nan, '').groupby('SUID').agg({
        'Series Date': lambda x: '/'.join(set(x)),
        'Start Time': 'min',
        'End Time': 'max',
        'Study Description': lambda x: '/'.join(set(x)),
        'Patient ID': lambda x: '/'.join(set(x)),
        'Machine Group': lambda x: '/'.join(set(x)),
        'Modality': lambda x: '/'.join(set(x)),
        'Protocol Name': lambda x: '/'.join(set(x))
    }).sort_values(['Series Date', 'Start Time', 'Machine Group', 'SUID'])\
    .rename(columns={'Series Date': 'Date'})
             
    # merge back into the main DataFrame
    if df_studies is None:
        df_studies = df_studies_for_day
    else:
        df_studies = pd.concat([df_studies, df_studies_for_day]).sort_values(['Series Date', 'Start Time', 'Machine Group', 'SUID'])

# save the newly extented studies to the studies file
if len(days_to_process) > 0
    df_studies.to_pickle(studies_save_path)

df_studies_query = df_studies.query('Date >= "{}" & Date <= "{}"'.format(start_date, end_date)).copy()
return df_studies_query

## Create some consensus on the studies descriptions

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-01'
config['main']['end_date'] = '2019-12-06'
df_studies = load_transform_and_save_data_from_files(config)

df_studies['short_descr'] = df_studies['Study Description'].str.lower().apply(lambda m: re.sub(r'[ _\-^()\+:\.\']', '', m))
df_studies['Description'] = None

# display(df_studies.groupby('Study Description')['Patient ID'].count().sort_values(ascending=False).iloc[0:10])
# display(df_studies.groupby('short_descr')['Patient ID'].count().sort_values(ascending=False).iloc[0:10])

for descr in config['description']:
    for descr_pattern in config['description'][descr].split(','):
        df_studies_match = df_studies[(df_studies['short_descr'].str.match('^' + descr_pattern + '$')) & (df_studies['Description'].isnull())]
        if len(df_studies_match) == 0:
            logging.info('descr [{:30s}] pattern "{:30s}": {:4d} matches'.format(descr, descr_pattern, len(df_studies_match)))
        df_studies.loc[df_studies.index.isin(df_studies_match.index), 'Description'] = descr

display(df_studies.groupby('Description')['Patient ID'].count().sort_values(ascending=False).iloc[0:10])
df_no_descr = df_studies[df_studies['Description'].isnull()][['Patient ID', 'short_descr', 'Study Description']].groupby(['Study Description', 'short_descr'])['Patient ID'].count().sort_values(ascending=False)[0:15].reset_index()
display(df_no_descr)
df_no_descr = df_no_descr.drop(columns='Patient ID')
df_no_descr['Study Description'] = df_no_descr['Study Description'].str.replace('_', ' ').str.replace(' \(Adulte?\)', '').str.replace('PET\^. ', '').str.upper()
abdo_rows = df_no_descr[df_no_descr['Study Description'].str.match('ABDOMEN')]
df_no_descr.loc[df_no_descr.index.isin(abdo_rows.index), 'Study Description'] = df_no_descr.loc[df_no_descr.index.isin(abdo_rows.index), 'Study Description'].str.replace('ABDOMEN\^. ', '') + ' ABDOMEN'
for ind in df_no_descr.index:
    print('{} = {}'.format(df_no_descr.loc[ind, 'Study Description'], df_no_descr.loc[ind, 'short_descr']))

### Sort the description patterns from the config

In [ ]:
descr_dict = {}
for descr in config['description']:
    descr_dict[descr] = []
    for descr_pattern in config['description'][descr].split(','):
        descr_dict[descr].append(descr_pattern)
for key in sorted(descr_dict.keys()):
    print('{} = {}'.format(key, ','.join(descr_dict[key])))

### Get the most common exams per machine

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-01'
config['main']['end_date'] = '2019-12-06'
df_studies = load_transform_and_save_data_from_files(config)

df_studies['short_descr'] = df_studies['Study Description'].str.lower().apply(lambda m: re.sub(r'[ _\-^()\+:\.\']', '', m))
df_studies['Description'] = None

# display(df_studies.groupby('Study Description')['Patient ID'].count().sort_values(ascending=False).iloc[0:10])
# display(df_studies.groupby('short_descr')['Patient ID'].count().sort_values(ascending=False).iloc[0:10])

for descr in config['description']:
    for descr_pattern in config['description'][descr].split(','):
        df_studies_match = df_studies[(df_studies['short_descr'].str.match('^' + descr_pattern + '$')) & (df_studies['Description'].isnull())]
        if len(df_studies_match) == 0:
            logging.debug('descr [{:30s}] pattern "{:30s}": {:4d} matches'.format(descr, descr_pattern, len(df_studies_match)))
        df_studies.loc[df_studies.index.isin(df_studies_match.index), 'Description'] = descr

        descr_dict = {}
for descr in config['description']:
    descr_dict[descr] = []
    for descr_pattern in config['description'][descr].split(','):
        descr_dict[descr].append(descr_pattern)
    
df_studies_clean = df_studies[df_studies['Machine Group'] != 'mixed cases'].copy()
df_studies_clean['Machine'] = df_studies_clean['Machine Group'].str.replace('NoCT', '')
df_descr_count = df_studies_clean.drop(columns='Machine Group').groupby(['Machine', 'Description'])['Patient ID'].count().reset_index()
df_descr_count = df_descr_count.rename(columns={'Patient ID': 'Count'})[['Machine', 'Description', 'Count']].sort_values('Count', ascending=False)
for machine in set(df_descr_count['Machine']):
    print('[description_{}]'.format(machine.lower().replace(' ', '')))
    for descr_key in sorted(descr_dict.keys()):
        if len(df_descr_count.query('Machine == @machine and Description == @descr_key')['Count']) > 0 \
                and int(df_descr_count.query('Machine == @machine and Description == @descr_key')['Count']) > 10:
            print('{} = {}'.format(descr_key, ','.join(descr_dict[descr_key])))

In [ ]:
int(df_descr_count.query('Machine == @machine and Description == @descr_key')['Count'])

### Load the config and get the series manually

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-07'
config['main']['end_date'] = '2019-04-26'

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-07'
config['main']['end_date'] = '2019-04-26'
df_series = load_data_from_files(config)

indices_to_exclude = []
logging.info('Found {} series before filtering description'.format(len(df_series)))
for descr_pattern in config['retrieve']['series_descr_patterns_to_exclude'].split('\n'):
    to_exclude_rows = df_series['Series Description'].str.match(descr_pattern, case=False)
    # gather all the indices
    indices_to_exclude.append(to_exclude_rows[to_exclude_rows == True].index)
# flatten the list
indices_to_exclude = [index for indices in indices_to_exclude for index in indices.values]
# if there is something to exclude, show a message and drop the rows
if len(indices_to_exclude) > 0:
    logging.info('Found {} series to exclude based on their description: "{}"'.format(len(indices_to_exclude),
        '", "'.join(df_series.loc[indices_to_exclude]['Series Description'])))
    df_series.drop(indices_to_exclude, inplace=True)
logging.info('Found {} series after filtering description'.format(len(df_series)))

# further filter out some Series that are not primary acquisitions (and do not contain any relevant time information)
df_series = df_series[~df_series['Protocol Name'].isin(config['retrieve']['series_protocols_to_exclude'].split('\n'))]
logging.debug('Found {} series after filtering protocol names'.format(len(df_series)))

#df_series = df_series[~df_series['Series Description'].isin(['Protocole patient', 'Enhancement curve'])]
#df_series = df_series[df_series.Machine != 'syngo.via.VB30A']
df_series

In [ ]:
df_series[df_series.Machine == 'syngo.via.VB30A']

### Do the marking of the re-takes

In [ ]:
df_series = mark_retakes(config, df_series)

### Do the grouping

In [ ]:
df_series, df_count_series, df_count_studies, df_count_series_day, df_count_study_day, df_count_study_weekday = do_series_groupby(config, df_series)

In [ ]:
df_count_studies

### Check what happened in the "mixed cases" studies

In [ ]:
with pd.option_context("display.max_rows", 20): display(df_series[df_series['Machine Group'] == 'mixed cases'])

In [ ]:
set(df_series[df_series['Machine Group'] == 'mixed cases']['Study Description'])

### Do some counting on different fields

In [ ]:
for field in ['Institution Name', 'Machine', 'Machine Group', 'Modality', 'Series Description', 'Study Description', 'Patient ID', 'i_take']:
    logging.info('Number of *Series* groupped by "{}"'.format(field))
    display(df_series.groupby(field)['SUID'].count())
    logging.info('Number of *Studies* groupped by "{}"'.format(field))
    display(df_series.groupby([field, 'SUID']).count().reset_index().groupby(field)['SUID'].count())
    logging.info('='*160)

### Figure out the start and end times of each study

In [ ]:
df_studies = df_series.dropna().groupby('SUID').agg({
    'Series Date': lambda x: '/'.join(set(x)),
    'Start Time': 'min',
    'End Time': 'max',
    'Study Description': lambda x: '/'.join(set(x)),
    'Machine Group': lambda x: '/'.join(set(x)),
    'Modality': lambda x: '/'.join(set(x)),
    'Institution Name': lambda x: '/'.join(set(x)),
    'Protocol Name': lambda x: '/'.join(set(x))
}).sort_values(['Series Date', 'Machine Group', 'Start Time', 'SUID'])
studies_save_path = 'data/studies/studies_{}_{}.pkl'.format(config['main']['start_date'], config['main']['end_date']).replace('-', '')
df_studies.to_pickle(studies_save_path)

In [ ]:
df_studies = load_transform_and_save_data_from_files(config)

### Group the series descriptions

In [ ]:
set(df_studies['Study Description'])